In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

let's extarct the IMDB Movie Data

In [ ]:
movies=pd.read_csv('../input/imdb-data/IMDB-Movie-Data.csv')

In [ ]:
movies.head(2)

Let us look at the Columns and Data Type.

In [ ]:
movies.info()
movies.describe()

From the above table it is clearly visible that Revenue has some of the movies which gained very high Revenue their is huge gap beetween 50% -75% - max value.

Also, from the above data colum it is understood, Revenue and Metascore column has missing values which needs to be treated.
But Revenue of the movie is very sensitive and which we can not relace by Mean, Median or Mode filling.

We can go ahead and drop those rows from our EDA process. 

In [ ]:
df=movies[~movies['Revenue (Millions)'].isnull()].copy()
df.info()

Since Metascore still has few null records left whcih can be replaced using mean value treatment.

In [ ]:
max_metascore =df['Metascore'].mean()
df.Metascore.fillna(max_metascore,inplace=True)

Now we can go ahead with analyzing our Table.

Let us first look at the Revenue of all the movies over period of time.


In [ ]:
sns.boxplot(data=df,x='Year', y='Revenue (Millions)')
plt.show()

We can observe from above plot, Year 2015 and 2009 clearly has datapoint with highest Revenue generating movie.

Which we will observe in our further analysis.

I will be adding a column with 1 in all the rows which will be usefull in our further analysis we wil do. Also, for now we will keep our original Table Sorted by Revenue.

In [ ]:
df['cnt']=1

In [ ]:
sns.pairplot(data=df, vars=["Votes", "Revenue (Millions)","Rating",
                            "Runtime (Minutes)","Metascore"])
plt.show()

The above Pairplot gives us the basic understanding on distribution of all the numerical column.

If we look at the Relationship of Revenue and Votes- YOY

In [ ]:
px.scatter(df,x='Revenue (Millions)',y='Votes',size='Votes', 
           color = 'Rating',hover_name='Title',animation_frame='Year',
           title="Distribution of Revenue and Votes")

From the above plot we observe that their are movies which has Higher Rating along with Higher Votes still Revenue is not up to the Mark.
Year - 2012 (Django Unchained)
Year - 2014 (Interstellar)
Year - 2006 (The Prestige,The Departed)

Let us extract more insights from the Table....
First split the Genre and Actor Table for more understandig !

In [ ]:
df[['Genre1', 'Genre2','Genre3']] = df['Genre'].str.split(',', 3, expand=True).fillna(value="")
df[['Actor1', 'Actor2','Actor3']] = df['Actors'].str.split(',', 2,expand=True).fillna(value="")

In [ ]:
df['Genre2']=df['Genre2'].fillna(df.Genre1, inplace=True)
df['Genre3']=df['Genre3'].fillna(df.Genre1,inplace=True)

In [ ]:
df_by_genre1 = df.groupby('Genre1')
df_by_genre2 = df.groupby('Genre2')
df_by_genre3 = df.groupby('Genre3')
agg_genre1=df_by_genre1.sum()
agg_genre2=df_by_genre2.sum()
agg_genre3=df_by_genre3.sum()
ge1_ge2 = agg_genre1.add(agg_genre2, fill_value=0.0)
final_genre = ge1_ge2.add(agg_genre3, fill_value=0.0)
genre_temp=final_genre.reset_index().head(10)
genre_temp

After grouping and adding all the Genre we observe that Adventure has the maximum Revenue and followed by Action.

In [ ]:
sns.set(style="darkgrid")

In [ ]:
fig, ax1 = plt.subplots(figsize = (14,6))

ax2 = ax1.twinx()
ax1.bar(genre_temp['Genre1'],genre_temp['Revenue (Millions)'])
ax2.plot(genre_temp['Genre1'],genre_temp['cnt'], 'o-', color="red" )
ax1.set_xlabel('Genre')
ax1.set_ylabel('Revenue', color='b')
ax2.set_ylabel('No of Movies', color='r')
plt.show()

Genre - Action has maximum numbers of movies and which generates maximum Revenue compared to other Genre.
But if we look at Drama and Comedy has on an average 150 nos of movies but generates very less revenue in comparision with other category of Genre and respective Revenues.

Now we will look at the number of (movies) released in year.....
YOY no of releases increaing linearly and year 2016 seems to have spike in no of relases. But Same growth is not observed for Revenue !

In [ ]:
df_year = df.groupby('Year')['cnt'].count().reset_index()
px.line(df_year, x='Year',y='cnt',template='plotly_dark',text='cnt',
        title="No of releases in Year")

In [ ]:
df_Rev = df.groupby('Year')['Revenue (Millions)'].sum().round(1).reset_index()
px.line(df_Rev, x='Year',y='Revenue (Millions)',template='plotly_dark',text='Revenue (Millions)',
        title="Revenue in Years")

We will now drill down within year 2016 - released movies..... !!

Top Revenue generating movies in year 2016 and their Ratings !

Captain America and Rouge One is most Revenue generating movies of Year 2016 being below 8 Rating.
and on the other hand Deadpool and Zootopia having Rating above 8 still the revenue Ranks at 4th and 5th. 

***One point to observe here Zootopia belongs to Genre - Animation and that can be one of reason of lagging behine in Revenue but still high in Rating.***

In [ ]:
df_2016 = df[df['Year']==2016].copy()
df_temp=df_2016.sort_values(by='Revenue (Millions)',ascending = False).head(10)
px.bar(df_temp,y='Revenue (Millions)',x='Title',color='Genre1',text='Rating',
       title='Most successful Movies/Genre of Year:2016')

In [ ]:
plt.figure(figsize=(10,7),facecolor='white')
sns.distplot(df['Runtime (Minutes)'],bins=10,kde=True)
plt.suptitle("Distribution of Movie by RunTime", fontsize=20)
plt.show()

**Most of Movies Belongs to Runtime Range - 90 to 130 Minutes !**

In [ ]:
movies_Rev=df.sort_values(by='Revenue (Millions)',ascending = False)[0:10]
px.pie(movies_Rev, values='Revenue (Millions)', names='Actor1',title='Actors vs Revenue Distribution')

In [ ]:
px.pie(movies_Rev, values='Revenue (Millions)', names='Title', title='Movies vs Revenue Distribution')

If We consider the Primary actor of the Movies and and Revenue Generated by respective movie is Robert Downey Jr.

But that is not the case, chances are on the above plot we distribution of Revenue is additing of one or more movies done by respective Actor.

We will further analyse on this part !!

In [ ]:
px.bar(movies_Rev,x='Actor1',y='Revenue (Millions)',color='Votes',
       hover_name='Title', text='Rating',template='plotly_dark')

Now from above Plot, it is clear that why Revenue generated by the Movies of Robert Downey Jr. is Highest amon the all because it has more then one movie.

Also, a intresting point to observe from above plot -> Movie - The Dark Knight got the Highest Rating of 9 and Highest Votes above 1.6M still below average Revenue generated by Movie copare to other Movie and actor! 

*Let us now look at the list of Top Directors and their critic score !*

In [ ]:
top_director= df.groupby('Director')['Revenue (Millions)'].sum().reset_index()
top_director.sort_values('Revenue (Millions)',ascending =False)[0:5]

In [ ]:
px.bar(movies_Rev,x='Director',y='Revenue (Millions)',
       color='Metascore',hover_name='Title',
       template='plotly_dark',text='Revenue (Millions)', title='Top Directors and Revenue Distribution by Metascore')

In [ ]:
movies_Votes=df.sort_values(by='Votes',ascending = False)[0:20]

px.bar(movies_Votes,x='Actor1',y='Votes',color='Revenue (Millions)',
       hover_name='Title',text='Rating',template='plotly_dark',
      title='Most Popular/Voted Actors')

In [ ]:
px.bar(movies_Votes,x='Title',y='Votes',color='Revenue (Millions)',
       hover_name='Title',text='Metascore',template='plotly_dark',
      title='Most Popular/Voted Movies')

We can extract list of Top Actors by Revenue generated in Movies.
And Let us look at the distribution of Revenue and Rating for Top Actors.

In [ ]:
top_Actor = pd.pivot_table(df,index=['Actor1'],
                           values=['Revenue (Millions)','Metascore','Votes','Rating','cnt'],
                           aggfunc={'Revenue (Millions)':np.sum,'Metascore':np.mean,
                                    'Votes':np.mean,'Rating':np.mean,'cnt':np.sum})

In [ ]:
actor = top_Actor.sort_values(by='Revenue (Millions)',
                              ascending = False).reset_index()[0:20]
actor

In [ ]:

px.scatter(actor,y= 'Revenue (Millions)',x= 'Rating', 
           color= 'cnt',size='Rating',text='Actor1',
           title='Revenue Vs Rating of most Popular Actor and No of Movies')

In [ ]:
px.histogram(df, x='Metascore',y='Revenue (Millions)',title='Impact of Metascore on Revenue')

We can observe from the above plot - Metascore from 55 to 70 seems to have highest Revenue ! 